In [1]:
#Import
import psycopg2
import re
import pandas as pd

In [2]:
#Connect to the database
def connect():
    return psycopg2.connect("dbname=cadastre_napoleonien_icdar24 user=annotator password=cadastre_annotator host=localhost port=5434")

IMG = "FRAD094_3P_001078"

In [3]:
#Get the list of tables of the schema with name = IMG
def get_tables(conn, img):
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s", (img,))
    tables = cur.fetchall()
    cur.close()
    return tables

get_tables(connect(), IMG)

[('document_regions',), ('map_text',)]

In [4]:
#In table map_text of schema with name = IMG, get the rows that have column "region_type" = parcel number
def get_parcel_numbers(conn, img):
    cur = conn.cursor()
    cur.execute(f'SELECT * FROM "{img}"."map_text"')
    parcel_numbers = cur.fetchall()
    cur.close()
    return parcel_numbers

In [5]:
_ = get_parcel_numbers(connect(), IMG)

In [6]:
#Convert into a pd df
import pandas as pd

def get_parcel_numbers_df(conn, img):
    return pd.DataFrame(get_parcel_numbers(conn, img))

df = get_parcel_numbers_df(connect(), IMG)

In [7]:
columns = ["fid","geom","groupid","wordid","region_type","transcription","force_ignore_evaluation","illegible","truncated","comment","checked"]
df.columns = columns

In [8]:
df

,fid,geom,groupid,wordid,region_type,transcription,force_ignore_evaluation,illegible,truncated,comment,checked
0,1,0103000020E6100000010000000900000091E07B667365...,1,1,non_toponym,L'hay,False,False,False,None,False
1,2,0103000020E6100000010000000B0000007586E93D8D37...,2,11,non_toponym,Section,False,False,False,None,False
2,3,0103000020E61000000100000005000000B85390CB9F76...,3,21,non_toponym,D,False,False,False,None,False
3,4,0103000020E610000001000000050000000234214775D7...,4,31,non_toponym,des,False,False,False,None,False
4,5,0103000020E61000000100000007000000A5713EDEBE28...,5,41,non_toponym,Roux,False,False,False,None,False
...,...,...,...,...,...,...,...,...,...,...,...
903,647,0103000020E610000001000000050000007DE519D9B14A...,593,6461,parcel_number,33,False,False,False,En réalité 533,False
904,681,0103000020E610000001000000050000002A4EE7FD1B27...,627,6801,parcel_number,68,False,False,False,En réalité 568,False
905,682,0103000020E61000000100000007000000D2DF949BC08E...,628,6811,parcel_number,69,False,False,False,En réalité 569,False
906,736,0103000020E61000000100000005000000A1575154F449...,682,7351,parcel_number,23,False,False,False,En réalité 623,False


## Check parcel numbers

In [9]:
parcel_numbers = df[df["region_type"] == "parcel_number"]

In [10]:
#Parcel numbers features that have no numbers in the transcription (to search for misclassified objects)
parcel_numbers_no_numbers = parcel_numbers[parcel_numbers["transcription"].str.contains(r'\d') == False]
parcel_numbers_no_numbers["transcription"].to_list()

[]

In [11]:
#Parcel numbers features that have both numbers and letters (to search for misclassified objects)
parcel_numbers_numbers_and_letters = parcel_numbers[parcel_numbers["transcription"].str.contains(r'[a-zA-Z]') & parcel_numbers["transcription"].str.contains(r'\d')]
parcel_numbers_numbers_and_letters["transcription"].to_list()

['21^{bis}',
 '38^{bis}',
 '58.^{bis}',
 '131^{bis}',
 '203^{bis}',
 '392.^{bis}',
 '427^{bis}',
 '486^{bis}']

In [12]:
#Get the list of duplicated numbers
def get_duplicated_parcel_numbers(parcel_numbers):
    return parcel_numbers[parcel_numbers.duplicated(subset=["transcription"])]["transcription"].unique()

duplicated_parcel_numbers = get_duplicated_parcel_numbers(parcel_numbers)
print(sorted(duplicated_parcel_numbers))

['10', '23', '24', '33', '39', '40', '48', '49', '56', '68', '69', '74', '75', '76', '82', '83', '86', '9', '98']


In [13]:
#Get the list of numbers that are at least 3 times or more in the list
def get_repeated_parcel_numbers(parcel_numbers):
    return parcel_numbers["transcription"].value_counts()[parcel_numbers["transcription"].value_counts() >= 3].index

repeated_parcel_numbers = get_repeated_parcel_numbers(parcel_numbers)
repeated_parcel_numbers

Index(['68', '69'], dtype='object', name='transcription')

In [14]:
# Step 1: Clean the column to retain only numerical characters
parcel_numbers['cleaned'] = parcel_numbers['transcription'].apply(lambda x: int(''.join(re.findall(r'\d+', x))) if re.findall(r'\d+', x) else None)

# Step 2: Drop any None values, convert to a list, and sort
numbers_list = sorted(parcel_numbers['cleaned'].dropna().tolist())

# Step 3: Identify missing numbers in the sequence
if numbers_list:
    full_range = range(numbers_list[0], numbers_list[-1] + 1)
    missing_numbers = sorted(set(full_range) - set(numbers_list))
else:
    missing_numbers = []

# Output results
print("Total Numbers in the table:", len(numbers_list))
print("Min Number:", numbers_list[0] if numbers_list else None)
print("Max Number:", numbers_list[-1] if numbers_list else None)
if numbers_list[-1] > (numbers_list[-1]-numbers_list[0]):
    print("ALERT! Something might be wrong (or not) with the numbers sequence.")
#print("Sorted Numbers List:", numbers_list)
print("Missing numbers in the sequence:", missing_numbers)

#Create ranges of missing numbers
def find_ranges(numbers):
    ranges = []
    for number in numbers:
        if not ranges or number != ranges[-1][-1] + 1:
            ranges.append([])
        ranges[-1].append(number)
    #for each range, create anew list only with the min and max value
    ranges = [range[0] if range[0] == range[-1] else (range[0], range[-1]) for range in ranges]
    return ranges
missing_ranges = find_ranges(missing_numbers)
print("Ranges of missing numbers:", missing_ranges)
print("Help:")
print("- In case of large ranges of missing numbers, these numbers might be in an other map of the same section.")
print("- Small ranges or unique numbers missing or more suceptibles to effectively have been missed or miss-transcribed (but not always :) ).")

Total Numbers in the table: 641
Min Number: 1
Max Number: 637
ALERT! Something might be wrong (or not) with the numbers sequence.
Missing numbers in the sequence: [139, 140, 168, 169, 186, 274, 275, 276, 282, 283, 298, 336, 352, 356, 448, 449, 502, 509, 510, 533, 552, 568, 569, 623, 624]
Ranges of missing numbers: [(139, 140), (168, 169), 186, (274, 276), (282, 283), 298, 336, 352, 356, (448, 449), 502, (509, 510), 533, 552, (568, 569), (623, 624)]
Help:
- In case of large ranges of missing numbers, these numbers might be in an other map of the same section.
- Small ranges or unique numbers missing or more suceptibles to effectively have been missed or miss-transcribed (but not always :) ).


C:\Users\stual\AppData\Local\Temp\ipykernel_19072\8360852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcel_numbers['cleaned'] = parcel_numbers['transcription'].apply(lambda x: int(''.join(re.findall(r'\d+', x))) if re.findall(r'\d+', x) else None)
